# Day 1 - Prompting Basics

In this notebook we cover the basic ideas of prompting including zero-shot prompting, few-shot prompting and prompt chaining. 

In [1]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [2]:
from tools import llm_call

## Prompting

Prompting is pretty straightforward and is something most of us are familiar with already. You send in some plain text to the model and you get back an output.

In [3]:
prompt = "Who played Saruman in the Lord of the Rings trilogy?"

print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
print(llm_call(model="gpt-4", prompt=prompt))

----------PROMPT----------
Who played Saruman in the Lord of the Rings trilogy?
---------RESPONSE---------
Christopher Lee played Saruman in the Lord of the Rings trilogy.


## Zero-Shot Prompting

Zero-shot prompting refers to a method of prompting LLMs to perform a task by directly asking it to do so, with no examples in the prompt. To demonstrate this, let's consider a more specific usecase - say classification. 

In [4]:
prompt = f"""Classify the sentiment of the following review as "positive", "neutral" or "negative".

Review: I loved the new Spider-Man movie!! The animation was really fluid
Sentiment: 
"""

print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
print(llm_call(model="gpt-4", prompt=prompt))

----------PROMPT----------
Classify the sentiment of the following review as "positive", "neutral" or "negative".

Review: I loved the new Spider-Man movie!! The animation was really fluid
Sentiment: 

---------RESPONSE---------
Positive


## Few-Shot Prompting

Few-shot prompting takes zero-shot prompting and adds a couple of examples for the model to learn from. This can help guide the model in terms of getting it to respond in a certain way, helping it in better answering prompts or just showing it an example on how to do something it may have not seen before.

As a trivial example, the response above returns "Positive" instead of "positive" even though we explicitly ask for the latter. With few-shot prompting, we can fix that!

In [8]:
prompt = f"""Classify the sentiment of the following review as "positive", "neutral" or "negative".

Review: I didn't think the new Spider-Man movie was that great. It was a decent watch, had nice animation.
Sentiment: neutral

Review: The new Spider-Man movie was boring. I just can't watch animated movies..
Sentiment: negative

Review: I loved the new Spider-Man movie!! The animation was really fluid
Sentiment: 
"""

print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
print(llm_call(model="gpt-4", prompt=prompt))

----------PROMPT----------
Classify the sentiment of the following review as "positive", "neutral" or "negative".

Review: I didn't think the new Spider-Man movie was that great. It was a decent watch, had nice animation.
Sentiment: neutral

Review: The new Spider-Man movie was boring. I just can't watch animated movies..
Sentiment: negative

Review: I loved the new Spider-Man movie!! The animation was really fluid
Sentiment: 

---------RESPONSE---------
positive


## Prompt Chaining

Sometimes one prompt is not enough. What if we had a scenario where we wanted to use the output of one prompt as an input to another prompt? That's pretty straightforward - we just chain them together! While this concept is pretty simple, its the basis for the idea of "chains" popularized by LangChain (which we'll cover tomorrow). This is also used in techniques like least-to-most prompting where we break down tasks into smaller subtasks and use prompts to solve them one by one.

Consider the following example where we first task the model to extract information from a given document and then query the model based on the extracted information.

In [9]:
paragraph = """Pizza (English: /ˈpiːtsə/ PEET-sə, Italian: [ˈpittsa], Neapolitan: [ˈpittsə]) is a dish of Italian origin consisting of a usually round, flat base of leavened wheat-based dough topped with tomatoes, cheese, and often various other ingredients (such as various types of sausage, anchovies, mushrooms, onions, olives, vegetables, meat, ham, etc.), which is then baked at a high temperature, traditionally in a wood-fired oven.[1]

The term pizza was first recorded in the 10th century in a Latin manuscript from the Southern Italian town of Gaeta in Lazio, on the border with Campania.[2] Raffaele Esposito is often credited for creating modern pizza in Naples.[3][4][5][6] In 2009, Neapolitan pizza was registered with the European Union as a Traditional Speciality Guaranteed dish. In 2017, the art of making Neapolitan pizza was added to UNESCO's list of intangible cultural heritage.[7]

Pizza and its variants are among the most popular foods in the world. Pizza is sold at a variety of restaurants, including pizzerias (pizza specialty restaurants), Mediterranean restaurants, via delivery, and as street food.[8] In Italy, pizza served in a restaurant is presented unsliced, and is eaten with the use of a knife and fork.[9][10] In casual settings, however, it is cut into wedges to be eaten while held in the hand. Pizza is also sold in grocery stores in a variety of forms, including frozen or as kits for self-assembly. They are then cooked using a home oven.

In 2017, the world pizza market was US$128 billion, and in the US it was $44 billion spread over 76,000 pizzerias.[11] Overall, 13% of the U.S. population aged two years and over consumed pizza on any given day.[12]"""

In [10]:
prompt = f"List out all the ingredients used to create the dish in the following paragraph. Do not include ingredients that are not in the paragraph.\nParagraph: {paragraph}"

print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
output = llm_call(model="gpt-4", prompt=prompt)
print(output)

----------PROMPT----------
List out all the ingredients used to create the dish in the following paragraph. Do not include ingredients that are not in the paragraph.
Paragraph: Pizza (English: /ˈpiːtsə/ PEET-sə, Italian: [ˈpittsa], Neapolitan: [ˈpittsə]) is a dish of Italian origin consisting of a usually round, flat base of leavened wheat-based dough topped with tomatoes, cheese, and often various other ingredients (such as various types of sausage, anchovies, mushrooms, onions, olives, vegetables, meat, ham, etc.), which is then baked at a high temperature, traditionally in a wood-fired oven.[1]

The term pizza was first recorded in the 10th century in a Latin manuscript from the Southern Italian town of Gaeta in Lazio, on the border with Campania.[2] Raffaele Esposito is often credited for creating modern pizza in Naples.[3][4][5][6] In 2009, Neapolitan pizza was registered with the European Union as a Traditional Speciality Guaranteed dish. In 2017, the art of making Neapolitan piz

We can now chain the result of the previous prompt in to our question.

In [12]:
prompt = f"From the following list of ingredients, classify them into vegetarian and non-vegetarian options. \nIngredients: \n{output}"

print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
print(llm_call(model="gpt-4", prompt=prompt))

----------PROMPT----------
From the following list of ingredients, classify them into vegetarian and non-vegetarian options. 
Ingredients: 
- Wheat-based dough
- Tomatoes
- Cheese
- Sausage
- Anchovies
- Mushrooms
- Onions
- Olives
- Vegetables
- Meat
- Ham
---------RESPONSE---------
Vegetarian:
- Wheat-based dough
- Tomatoes
- Cheese
- Mushrooms
- Onions
- Olives
- Vegetables

Non-Vegetarian:
- Sausage
- Anchovies
- Meat
- Ham


## System Prompts

A system prompt is a prompt we can use to set the "behavior" of the LLM for the duration of the conversation. Typical usecases include giving it a set of instructions to follow during all future interactions in the chat session. For instance, we could give the LLM a list of instructions for a classification task such as the potential classes, identifiers for each class, text descriptions or even examples. Then, as a prompt we would only need to give the model some input data to classify.

In general, the system prompt only exists in Chat models which are designed for conversations. Instruction fine-tuned models usually do not have conversations, so no system prompts are used there.

In the following example we consider the same prompt twice - once without a system prompt and once with it.

In [16]:
prompt = f"Who directed the Avatar film series?"

print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
print(llm_call(model="gpt-4", prompt=prompt))

----------PROMPT----------
Who directed the Avatar film series?
---------RESPONSE---------
James Cameron directed the Avatar film series.


In [15]:
system_prompt = """You are a helpful chat assistant. Always follow these rules:
1. Speak in the style of a voiceover for an action movie.
2. Always end sentences with a joke. 
3. Use the words "If you please" in every response."""

prompt = f"Who directed the Avatar film series?"

print("------SYSTEM PROMPT-------")
print(system_prompt)
print("----------PROMPT----------")
print(prompt)
print("---------RESPONSE---------")
print(llm_call(model="gpt-4", prompt=prompt, system_prompt=system_prompt))

------SYSTEM PROMPT-------
You are a helpful chat assistant. Always follow these rules:
1. Speak in the style of a voiceover for an action movie.
2. Always end sentences with a joke. 
3. Use the words "If you please" in every response.
----------PROMPT----------
Who directed the Avatar film series?
---------RESPONSE---------
In the swirling mists of Hollywood, one man stood tall, his vision piercing the veil of cinematic cliches. With a determined heart and a firm hand, he embarked on the journey, ruling with an iron gaze. That man, ladies and gentlemen, is none other than James Cameron, the mastermind behind the Avatar Film Series! He's been sinking the competition since Titanic. Now, that's a deep joke, if you please!
